In [11]:
# Re-import necessary libraries since execution state was reset
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Simulate a 30-day time series with a separate timestamp column
date_rng = pd.date_range(start="2025-01-01", periods=30*24*60*60, freq="S")
df = pd.DataFrame({"timestamp": date_rng, "value": np.cumsum(np.random.randn(len(date_rng)))})

# Add additional irrelevant columns to mimic the real dataset
df["other_metric1"] = np.random.randn(len(df))
df["other_metric2"] = np.random.randn(len(df))

# Define rolling windows in seconds
rolling_windows = {
    "30min": 30 * 60,
    "1hr": 60 * 60,
    "4hr": 4 * 60 * 60,
    "1day": 24 * 60 * 60
}

# Compute rolling statistics
for name, window in rolling_windows.items():
    df[f"mean_{name}"] = df["value"].rolling(window=window, min_periods=1).mean()
    df[f"std_{name}"] = df["value"].rolling(window=window, min_periods=1).std()
    df[f"zscore_{name}"] = (df["value"] - df[f"mean_{name}"]) / df[f"std_{name}"]

# Convert timestamp column to datetime for filtering
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Select a recent 3-day moving window for visualization with relevant columns only
columns_to_keep = ["timestamp", "value", "mean_1hr", "mean_4hr", "mean_1day", "zscore_1hr"]
df_subset = df[(df["timestamp"] >= "2025-01-28") & (df["timestamp"] <= "2025-01-30")][columns_to_keep]


In [ ]:
fig1 = go.Figure()

# Add traces for value and rolling means
fig1.add_trace(go.Scatter(x=df_subset["timestamp"], y=df_subset["value"], mode="lines", name="Value", opacity=0.6))
fig1.add_trace(go.Scatter(x=df_subset["timestamp"], y=df_subset["mean_1hr"], mode="lines", name="Rolling Mean (1hr)", line=dict(dash="dash")))
fig1.add_trace(go.Scatter(x=df_subset["timestamp"], y=df_subset["mean_4hr"], mode="lines", name="Rolling Mean (4hr)", line=dict(dash="dash")))
fig1.add_trace(go.Scatter(x=df_subset["timestamp"], y=df_subset["mean_1day"], mode="lines", name="Rolling Mean (1 day)", line=dict(dash="dot")))

# Update layout for rolling mean plot
fig1.update_layout(
    title="Rolling Mean Analysis (Last 3 Days)",
    xaxis_title="Timestamp",
    yaxis_title="Value",
    xaxis=dict(tickangle=45),
    legend=dict(x=1.05, y=1, bordercolor="Black", borderwidth=1, traceorder="normal"),
    margin=dict(r=200)  # Add right margin to avoid overlap
)

# Show fig1ure
fig1.show()

In [ ]:
# Create Plotly figure for Z-score analysis
fig2 = go.Figure()

# Add Z-score trace
fig2.add_trace(go.Scatter(x=df_subset["timestamp"], y=df_subset["zscore_1hr"], mode="lines", name="Z-score (1hr)"))

# Add threshold lines for Z-score
fig2.add_trace(go.Scatter(x=df_subset["timestamp"], y=[2]*len(df_subset), mode="lines", name="Z=2 Threshold", line=dict(color='red', dash="dash")))
fig2.add_trace(go.Scatter(x=df_subset["timestamp"], y=[-2]*len(df_subset), mode="lines", name="Z=-2 Threshold", line=dict(color='red', dash="dash")))

# Update layout for Z-score plot
fig2.update_layout(
    title="Z-score Analysis (Last 3 Days)",
    xaxis_title="Timestamp",
    yaxis_title="Z-score",
    xaxis=dict(tickangle=45),
    legend=dict(x=1.05, y=1, bordercolor="Black", borderwidth=1, traceorder="normal"),
    margin=dict(r=200)  # Add right margin to avoid overlap
)

fig2.show()